# Endgame

Das Endspiel nutzt sogenannte *Syzygy tablebases*, die zu jedem möglichen Spielstand $ \mathcal{B} $ für bis zu 7 Figuren Informationen über die Metriken WDL (**Winn/Draw/Loss**) und DTZ (**Depth to Zero**) bereitstellen. Dabei sind `WDL` und `DTZ` Bewertungsfunktionen, die wie folgt definiert sind

$$ \texttt{WDL}: \mathcal{B} \rightarrow v $$

mit $ v \in \{ -2, -1, 0, 1, 2 \} $ und

$$ \texttt{DTZ}: \mathcal{B} \rightarrow w $$

mit $ w \in \{ -100, \dots, -1, 0, 1, \dots, 100 \} $.

## 50-Züge-Regel

Die 50-Züge-Regel beim Schach besagt, dass eine Partie dann als Remis gewertet werden kann, wenn in den letzten 50 aufeinanderfolgenden Spielzügen weder eine Figur geschlagen, noch ein Bauer gezogen wurde.

## WDL

In `WDL`-Dateien (Dateiendung `.rtbw`) sind Informationen zu Gewinn, Remis und Verlust unter Berücksichtigung der 50-Züge-Regel gespeichert. Auf diese Informationen kann während der Suche zugegriffen werden.

Dabei wird den Elementen der Menge $ \{ -2, -1, 0, 1, 2 \} $ die im Folgenden definierte Eigenschaft zugeordnet

* `2` die ziehende Seite gewinnt
* `0` es liegt ein Remis vor
* `-2` die ziehende Seite verliert
* `1` bei einem 'cursed win'
* `-1` bei einem 'blessed loss'

## DTZ
`DTZ`-Dateien (Dateiendung `.rtbz`) enthalten Informationen über die *Lagenzählung* für den Zugriff zu Beginn der Suche. Um Speicherplatz zu sparen gibt es in Anlehnung an die 50-Züge-Regel auch die Metrik `DTZ50`, die lediglich für eine Spielseite Informationen speichert. Für jede mögliche Position repräsentiert die DTZ die Anzahl der Züge des Gewinners bis zum Sieg. Hierfür werden folgende zwei Annahmen gemacht

* Gewinner minimiert die DTZ
* Verlierer maximiert die DTZ

## Zusammenhang von WDL und DTZ

Jedes Endspiel nutzt ein Paar dieser Informationen zur Evaluation des besten nächsten Zugs.

Nachstehende Tabelle beschreibt zusammengefasst das Verhalten der Funktionen.

| WDL | DTZ             | Beschreibung                                                                                                                                                                        |
| --- | --------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| -2  | -100 <= n <= -1 | Unbedingter Verlust (unter der Annahme, dass der 50-Züge-Zähler Null ist), wobei ein Nullzug in -n Zügen erzwungen werden kann.                                                     |
| -1  | n < -100        | Verlust, aber Remis nach der 50-Züge-Regel. Ein Nullzug kann in -n Zügen oder -n - 100 Zügen erzwungen werden (wenn eine spätere Phase für den 'blessed loss' verantwortlich ist).  |
|  0  | 0               | Remis.                                                                                                                                                                              |
|  1  | 100 < n         | Sieg, aber Unentschieden nach der 50-Züge-Regel. Ein Nullzug kann in n Zügen oder n - 100 Zügen erzwungen werden (wenn eine spätere Phase für den 'cursed win' verantwortlich ist). |
|  2  | 1 <= n <= 100   | Unbedingter Sieg (unter der Annahme, dass der 50-Züge-Zähler Null ist), wobei ein Nullzug in n Zügen erzwungen werden kann.                                                         |

Um den bestmöglichen nächsten Zug durch die Endspielbibliothek erhalten zu können, definieren wir den $ \texttt{Folge-Zug} $.

Ein Folge-Zug $ \mathcal{F} $ ist ein Tripel der Form

$$ \mathcal{F} = \langle M, W, D \rangle $$

wobei

- $ M $ ein Zug,
- $ W $ der der Position nach dem Zug $ M $ über die Metrik $ \texttt{WDL} $ zugeordnete Wert,
- $ D $ der der Position nach dem Zug $ M $ über die Metrik $ \texttt{DTZ} $ zugeordnete Wert ist.

Benutzerdefinierte Typ-Definition eines Folge-Zugs.

In [ ]:
FollowingMove = Tuple[chess.Move, int, int]

Die Funktion `get_metrics_from_next_legal_moves` nimmt als Argument eine Liste von Zügen $ \textit{Moves} $, berechnet für diese die Metriken WDL und DTZ und gibt eine Liste von Folge-Zügen $ \mathcal{F} $ zurück.

In [ ]:
def get_metrics_from_next_legal_moves(moves: List[chess.Move]) -> List[FollowingMove]:
    next_moves = []
    for move in moves:
        board.push(move)
        wdl_after_next_move = end_game_reader.probe_wdl(board)
        dtz_after_next_move = end_game_reader.probe_dtz(board)
        board.pop()
        next_moves.append((move, wdl_after_next_move, dtz_after_next_move))
    return next_moves

Die Funktion `get_relevant_moves` nimmt eine Liste von Folge-Zügen $ \mathcal{F} $ sowie eine optionale WDL und gibt alle Züge, gefiltert nach dem absoluten Wert der WDL zurück.

In [ ]:
def get_relevant_moves(moves: List[FollowingMove], wdl=-2) -> List[FollowingMove]:
    return [ (m, w, abs(d)) for m, w, d in moves if w == wdl ]

Die Funktion `select_best_move_tuple_from_zeroing` nimmt als Argument eine Liste von Folge-Zügen $ \mathcal{F} $ und prüft, ob einer dieser Züge die Metrik DTZ auf Null setzt, also ein Schlag- oder Bauernzug ist und gibt diesen zurück. Alternativ gibt diese Funktion den Zug mit der kleinsten DTZ zurück.

In [ ]:
def select_best_move_tuple_from_zeroing(moves_tpl: List[FollowingMove]) -> FollowingMove:
    for move_tpl in moves_tpl:
        if board.is_zeroing(move_tpl[0]):
            return move_tpl
    return min(moves_tpl, key=lambda move: move[2])

Die Funktion `get_next_end_game_move` wird aus der `main.ipynb` aufgerufen und berechnet mithilfe der zuvor vorgestellten Hilfsfunktionen den besten Zug für die jeweils aktuelle Seite.

In [ ]:
def get_next_end_game_move() -> chess.Move:
    # Check if the game is over due to checkmate, stalemate, insufficient material, ...
    if board.is_game_over():
        return None
    
    # Get the WDL from the currently given board
    wdl = end_game_reader.probe_wdl(board)
    # Get the DTZ from the currently given board
    dtz = end_game_reader.probe_dtz(board)
    
    # Get list of metrics from all legal moves
    moves = get_metrics_from_next_legal_moves(board.legal_moves)
    
    # Board.turn is winning
    if wdl == 2:
        # Get all relevant moves by filtering by WDL
        relevant_moves = get_relevant_moves(moves)
        
        # Get best move as minimum from list of Folge-Züge with given DTZ as key
        best_move = select_best_move_tuple_from_zeroing(relevant_moves)[0]
    # Board.turn is losing:
    elif wdl == -2:
        # Select best move by filtering by max DTZ
        best_move = max(moves, key=lambda move: move[2])[0]
    # Board is a draw
    elif wdl == 0:
        # Select move by filtering by WDL
        relevant_moves = get_relevant_moves(moves, 0)
        best_move = relevant_moves[0][0]
    else:
        # Some error occurs
        print(f'wdl has unknown value in get_next_end_game_move: wdl = {wdl}')
        sys.exit(1)
    
    return best_move

## Quellen

* https://www.chessprogramming.org/Syzygy_Bases
* https://www.chessprogramming.org/Endgame_Tablebases